# Funktionen zum Durchsuchen und Filtern der Reden




In [1]:
import json
import jsonlines

# Das (natural language toolkit) müsst ihr bestimmt installieren.
# Weiß noch jemand, wie das geht?
import nltk

import spacy
nlp = spacy.load('de') #load spacy model

# Die collections.Counter package müsst ihr bestimmt auch installieren.
# Die brauchen wir später, um Worte zu zählen.
from collections import Counter



# Hier legen wir fest, welche Daten (Wahlperiode 19 oder 20) wir laden:
legislatur = 20

# Wir generieren eine leere Liste:
alleReden = []

# Wir öffnen den entsprechende File (Dateipfad anpassen!):
with jsonlines.open(f'../../data/speeches_{legislatur}.jsonl') as f:
    for line in f.iter():
    # Wir packen alles Zeile für Zeile zu unserer Liste:
        alleReden.append(line)
        
# Wir sortieren nach Datum:
alleReden.sort(key = lambda x:x['date'])

# Wir lassen uns zeigen, wie viele Reden enthalten sind.
print(f'Die Liste enthält {len(alleReden)} Reden')
# Neue Zeile auf der Konsole:
print(f'\n')


Die Liste enthält 10791 Reden




## Suchfunktionen



In [2]:

## Zunächst brauchen wir eine Funktion, die uns die Reden gibt, die ein bestimmtes Wort enthalten. 
#  Funktion für Textsuche: 
#  Gibt eine Untermenge an Reden zurück, die einen bestimmten String (Wort) enthalten.

def find_speeches_with_word(search_term, speeches):
    filtered_speeches = []
    for speech in speeches:
        if ( search_term in speech['text'] ):
            filtered_speeches.append(speech)
    return filtered_speeches

## Probieren wir das mal aus.
#  Das ist unser Suchwort (oder String):
wort = 'letzte Generation'
#wort = 'Kapital'

#  Hier rufen wir die Suchfunktion auf und speichern die Untermenge der Reden.
untermenge = find_speeches_with_word(wort, alleReden)

# Wir lassen uns zeigen, wie viele Reden enthalten sind.
print(f'Diese Liste (Suche nach {wort}) enthält {len(untermenge)} Reden.')
# Neue Zeile auf der Konsole:
print(f'\n')

## Reden sind lang und die Worte tauchen in verschiedenen Kontexten auf.
#  Wir würden gerne alle Sätze sehen, in denen der Suchbegriff vorkommt.
#  Aber natürlich kommt unser Suchstring nur in Sätzen vor, die in de Untermenge an Reden sind. 

def find_sentences_with_word(search_term, speeches):
    sents_with_words = []
    for speech in speeches:
        sent_list = nltk.sent_tokenize(speech['text'])
        for sent in sent_list:
            if search_term in sent:
                sents_with_words.append(sent)
    return sents_with_words
                
# Probieren wir diese Funktion einmal aus:
satz_liste = find_sentences_with_word(wort,untermenge)

# Wir lassen uns zeigen, wie viele Sätze in der Liste enthalten sind.
print(f'Diese Liste (Suche nach {wort}) enthält {len(satz_liste)} Sätze')
# Neue Zeile auf der Konsole:
print(f'\n')

# Wollen wir uns die alle anzeigen lassen? Ja oder Nein?
wir_wollen = False
if wir_wollen:
    for satz in satz_liste:
        print(satz)

        
## Nun wäre es doch spannend, die Reden einer Partei oder eines Politikers zu sehen.
#  Dazu entwickeln wir eine Funktion, die es erlaubt, in den anderen Felder (keys) zu suchen.
#  Funktion, mit der man eine Menge an Reden nach verschiedenen Kriterien filtern kann.
#  Es wird die entsprechende Untermenge zurückgegeben.
#  'what' enthält den Key, wo gesucht werden soll. Interessant vor allem: 'name' und 'party'

def filter_speeches_for(what, search_term, speeches):
    filtered_speeches = []
    for speech in speeches:
        if search_term in speech[what]:
            filtered_speeches.append(speech)
        
    filtered_speeches.sort(key = lambda x:x['date'])   
    return filtered_speeches

# Beispiel: Für alle Reden von Olaf Scholz:
suche_nach = 'Olaf Scholz'
untermenge = filter_speeches_for('name', suche_nach, alleReden)
# Wir lassen uns zeigen, wie viele Reden enthalten sind.
print(f'Diese Liste (Suche nach {suche_nach}) enthält {len(untermenge)} Reden')
print(f'\n')

####
## Jetzt könnten wir die Sätze von Olaf Scholz mit einem bestimmten Wort anschauen.
#  Das ist unser Suchwort (oder String):
#wort = 'Klimawandel'
wort = 'Klimakrise'

#  Hier rufen wir die Suchfunktion auf und speichern die Untermenge der Reden.
untermenge = find_speeches_with_word(wort, alleReden)

# Wir suchen nach:
suche_nach = 'Olaf Scholz'
untermenge = filter_speeches_for('name', suche_nach, untermenge)
satz_liste = find_sentences_with_word(wort,untermenge)

print(f'In {len(untermenge)} Reden gibt es {len(satz_liste)} Sätze von {suche_nach}, die {wort} enthalten:')
print(f'\n')

# Die schauen wir uns an.
for sx,satz in enumerate(satz_liste):
    print(f' Satz {sx+1}: {satz}')
    print(f'\n')


Diese Liste (Suche nach letzte Generation) enthält 2 Reden.


Diese Liste (Suche nach letzte Generation) enthält 2 Sätze


Diese Liste (Suche nach Olaf Scholz) enthält 171 Reden


In 4 Reden gibt es 6 Sätze von Olaf Scholz, die Klimakrise enthalten:


 Satz 1: Die Klimakrise erfordert entschlossenes, systematisches und international abgestimmtes Vorgehen.


 Satz 2: Deshalb bieten wir China Zusammenarbeit an bei Menschheitsherausforderungen wie der Klimakrise, der Pandemie oder der Rüstungskontrolle.


 Satz 3: Vor uns liegen enorme Aufgaben: die wirtschaftliche Transformation voranbringen, die Klimakrise in den Griff bekommen, demografischen Wandel gestalten, Frieden in Europa sichern.


 Satz 4: Aber jetzt ist klar: Die Erneuerbaren brauchen wir nicht nur wegen der Klimakrise.


 Satz 5: Angesichts von Krieg und Klimakrise geht es heute auch um Europa selbst.


 Satz 6: Meine Damen und Herren, wir leben in einer besonders herausfordernden Zeit: Die Klimakrise spitzt sich zu; der Krie

In [6]:
#from nltk.corpus import stopwords

german_stop_words = nltk.corpus.stopwords.words('german')

####
## Lass mal was versuchen:
#wort = 'Kapitalismus'
#wort ='Nachhaltigkeit'

wort ='Klima'

#  Hier rufen wir die Suchfunktion auf und speichern die Untermenge der Reden.
untermenge = find_speeches_with_word(wort, alleReden)
satz_liste = find_sentences_with_word(wort,untermenge)

#print(f'In {len(untermenge)} Reden gibt es {len(satz_liste)} Sätze von {suche_nach}, die {wort} enthalten:')
print(f'In {len(untermenge)} Reden gibt es {len(satz_liste)} Sätze, die {wort} enthalten:')
print(f'\n')

# Die schauen wir uns an und generieren eine Wortliste
wort_liste = []
for sx,satz in enumerate(satz_liste):
    #print(f' Satz {sx+1}: {satz}')
    worte = nltk.word_tokenize(satz)
    #print(f' Satz {sx+1}: {worte}')
    wort_liste.extend(worte)

wort_liste_clean = []
for wort in wort_liste:
    if wort not in german_stop_words:
        if len(wort)>3:
            wort_liste_clean.append(wort)
    
print(f'In der Liste befinden sich {len(wort_liste_clean)} Worte: \n')
#print(wort_liste_clean)

from collections import Counter
counts = Counter(wort_liste_clean)
print(counts.most_common(20))



In 2072 Reden gibt es 4906 Sätze, die Klima enthalten:


In der Liste befinden sich 54076 Worte: 

[('Klimaschutz', 1245), ('Klimakrise', 509), ('Klimawandel', 388), ('Klima', 382), ('müssen', 369), ('mehr', 340), ('Klima-', 262), ('Euro', 228), ('Deutschland', 220), ('Menschen', 210), ('Klimaziele', 209), ('beim', 198), ('geht', 193), ('erreichen', 181), ('Milliarden', 171), ('Klimawandels', 169), ('ganz', 168), ('brauchen', 163), ('schon', 162), ('unserer', 160)]


In [ ]:


####
## Lass mal was versuchen:
wort = 'Kapitalismus'
#wort ='Nachhaltigkeit'
wort='Krise'

#  Hier rufen wir die Suchfunktion auf und speichern die Untermenge der Reden.
untermenge = find_speeches_with_word(wort, alleReden)

suche_nach = 'Olaf Scholz'
untermenge = filter_speeches_for('name', suche_nach, alleReden)

satz_liste = find_sentences_with_word(wort,untermenge)

print(f'In {len(untermenge)} Reden gibt es {len(satz_liste)} Sätze von {suche_nach}, die {wort} enthalten:')
#print(f'In {len(untermenge)} Reden gibt es {len(satz_liste)} Sätze, die {wort} enthalten:')
print(f'\n')

    



In [21]:


# Die schauen wir uns an und generieren eine Wortliste
def get_list_of_lemmata(satz_liste):
    wort_liste = []
    for sx,satz in enumerate(satz_liste):
        #print(f' Satz {sx+1}: {satz}')
        worte = nltk.word_tokenize(satz)
        #print(f' Satz {sx+1}: {worte}')
        wort_liste.extend(worte)

    wort_liste_clean = []
    for wort in wort_liste:
        if wort not in german_stop_words:
            if len(wort)>3:
                wort_liste_clean.append(wort)
    return wort_liste_clean
        

# Daselbe mit Spacy
def get_list_of_lemmata_spacy(satz_liste):
    wort_liste = []
    for sx,satz in enumerate(satz_liste):
        #print(f' Satz {sx+1}: {satz}')
        doc = nlp(satz)
        for token in doc:
            if token.pos_ in ['NOUN']:
                #print(token.lemma_)
                wort_liste.append(token.lemma_)    
    return wort_liste
    
#print(wort_liste)    

counts = Counter(wort_liste).most_common(20)
print(counts[0][0])

,


In [35]:
###
# Let's try an iterative process:

import random

search_depth = 20
sentence_limit = 20400
scope = 5

#suche_nach = 'SPD'
#reden = filter_speeches_for('party', suche_nach, alleReden)
reden = alleReden

nodes = []
neighborhoods = {}

start_wort = 'Kapitalismus'
start_wort ='Nachhaltigkeit'
start_wort ='Krise'
start_wort ='Demokratie'
start_wort ='Plattform'
start_wort ='Medien'
start_wort ='Klima'
#start_wort ='müssen'

partei = 'AfD'

nodes.append(start_wort)
for step in range(0,search_depth):
    print('\n')
    print(f'Search step {step+1}: {nodes[step]} \n')
    wort = nodes[step]
    untermengeP = filter_speeches_for('party', partei, reden)
    untermengeW = find_speeches_with_word(wort, untermengeP)
    if len(untermengeW)/len(untermengeP) < 0.6:
        satz_liste = find_sentences_with_word(wort,untermengeW)
        print(f'--> Die Suche nach {wort} ergab {len(untermengeW)} Reden')
        if(len(satz_liste) > sentence_limit):
            print(f'--> Die Suche nach {wort} ergab {len(satz_liste)} Sätze')
            neighborhoods.update({wort : []})
            #satz_liste = random.sample(satz_liste,sentence_limit)
        else:
            wort_liste = get_list_of_lemmata_spacy(satz_liste)
            
            counts = Counter(wort_liste).most_common(scope)
            neighbors = []
            for ele in counts:
                new_word = ele[0]
                if new_word != wort and ele[1]>1:
                    neighbors.append(new_word)
                    if new_word not in nodes:
                        nodes.append(new_word)
                        print(f'Add {new_word}')
                    neighborhoods.update({wort : neighbors})
                else:
                    neighborhoods.update({wort : []})
    else:
        neighborhoods.update({wort : []})

#print(nodes)            



Search step 1: Klima 

--> Die Suche nach Klima ergab 230 Reden
Add Klimaschutz
Add Klimawandel
Add Euro
Add Herr


Search step 2: Klimaschutz 

--> Die Suche nach Klimaschutz ergab 50 Reden
Add Bundesregierung
Add Klimaschutzes
Add Artenschutz
Add Energie


Search step 3: Klimawandel 

--> Die Suche nach Klimawandel ergab 30 Reden
Add Klimawandels
Add Million
Add Anpassung


Search step 4: Euro 

--> Die Suche nach Euro ergab 631 Reden
Add Milliarde
Add Jahr


Search step 5: Herr 



Search step 6: Bundesregierung 

--> Die Suche nach Bundesregierung ergab 455 Reden
Add Land


Search step 7: Klimaschutzes 

--> Die Suche nach Klimaschutzes ergab 5 Reden


Search step 8: Artenschutz 

--> Die Suche nach Artenschutz ergab 13 Reden
Add Artenschutzes
Add Energiewende


Search step 9: Energie 

--> Die Suche nach Energie ergab 434 Reden
Add Energiepolitik
Add Energieversorgung
Add Energiepreise


Search step 10: Klimawandels 

--> Die Suche nach Klimawandels ergab 6 Reden
Add Bekämpfung


In [32]:
print(neighborhoods.keys())
print(nodes)

dict_keys(['Klima', 'Klimaschutz', 'Klimawandel', 'Euro', 'Herr', 'Bundesregierung', 'Klimaschutzes', 'Artenschutz', 'Energie', 'Klimawandels', 'Million', 'Anpassung', 'Milliarde', 'Jahr', 'Land', 'Energiewende', 'Energiepolitik', 'Energieversorgung', 'Energiepreise'])
['Klima', 'Klimaschutz', 'Klimawandel', 'Euro', 'Herr', 'Bundesregierung', 'Klimaschutzes', 'Artenschutz', 'Energie', 'Klimawandels', 'Million', 'Anpassung', 'Milliarde', 'Jahr', 'Land', 'Artenschutzes', 'Energiewende', 'Energiepolitik', 'Energieversorgung', 'Energiepreise', 'Bekämpfung', 'Mensch', 'Dame', 'Prozent', 'Landwirt', 'Bürger', 'Welt', 'Regierung', 'Winter', 'Kernkraft', 'Energiepreisen', 'Inflation']


In [36]:

import networkx as nx
import tqdm

graphforgephi = nx.Graph()

for ix,node in tqdm.tqdm(enumerate(nodes)):
    #graphforgephi.add_node(ix+1,name = node);
    graphforgephi.add_node(node,name = node);
    
print('nodes done')

for ix,node in tqdm.tqdm(enumerate(nodes[0:search_depth])):
    for neighbor in neighborhoods[node]:
        graphforgephi.add_edge(node,neighbor,weight=1)
print('links done')   
nx.write_gexf(graphforgephi, "graphforgephi.gexf")
print('save done')


32it [00:00, 112598.77it/s]
20it [00:00, 41568.92it/s]

nodes done
links done
save done
